## Exercise 1
#### You are given a dataset having more variables than observations. Assuming that there seems to be a linear relationship between the target variable and the input variables in the dataset, why ordinary least squares (OLS) is a bad option to estimate the model parameters? Which technique would be best to use? Why?

Ordinary least squares (OLS) is a bad option because when it is given a dataset having more variables than observtions, it becomes a math algebra problem because the model cannot estimate the extra coefficients. I would use Ridge Regression, which the coefficients are estimated by minimizing a slightly different quantity. In this model, the coefficient estimates are the values that minimize the error. 

## Exercise 2
#### For Ridge regression, if the regularization parameter, λ, is equal to 0, what are the implications?

(a) Large coefficients in the linear model are not penalized.\
(c) The objective function is the same as ordinary least squares objective function.

Answer: (f)  (a) and (c)

## Exercise 3
####  For Lasso Regression, if the regularization parameter, λ, is very high, which options are true? Select all that apply.

(a) Can be used to select important features of a dataset.\
(b) Shrinks the coefficients of less important features to exactly 0.

Answer: (f) (a) and (b)

## Exercise 4
#### Suppose you are using Ridge Regression and you notice that the training error and the validation error are almost equal and fairly high. Would you say that the model suffers from high bias or high variance? Should you increase the regularization parameter, λ, or reduce it?

When there is a training error and the validation error are almost equal and fairly high, I would say the model suffers of high bias, which is caused by underfitting the model. I would reduce the lambda to make the model more complex fitting it more.

## Exercise 5

In [1]:
# (a)
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LassoCV, LinearRegression, Ridge, RidgeCV
from sklearn.model_selection import KFold

# Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'gabrielferreira-data-455-bucket'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
file_key = "CarPrice_Assignment.csv"

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the csv file
car_price = pd.read_csv(file_content_stream)
car_price.head(1)

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0


In [2]:
# (b)
from sklearn.model_selection import train_test_split

# Defining input and target variables
X = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg']]
Y = car_price['price']

# Splitting the data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [3]:
# Selecting lasso as variable selector
from sklearn.linear_model import Lasso, LassoCV

# Estimating optimal lambda with lasso
lasso_cv = LassoCV(normalize = True, cv = 5).fit(X_train, Y_train)
lasso_cv = lasso_cv.alpha_

# Selecting variables with the above Lasso optimal lambda.
lasso_md = Lasso(alpha = lasso_cv, normalize = True).fit(X_train, Y_train)
lasso_md.coef_

array([208.65112006, 102.82824753, 290.205738  ,  65.41928608,
         1.6259893 , -21.71189354, -10.22755194])

In [4]:
# Repeating the above proccess 1000 times.

lasso_results = []

for i in range(0,1000):
    
    # Splitting the data into train and test
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    # Estimating optimal lambda with lasso
    lasso_cv = LassoCV(normalize = True, cv = 5).fit(X_train, Y_train)
    lasso_cv = lasso_cv.alpha_

    # Selecting variables with the above Lasso optimal lambda.
    lasso_md = Lasso(alpha = lasso_cv, normalize = True).fit(X_train, Y_train)
    lasso_results.append(lasso_md.coef_)

# Making a dataset for the results
lasso_coef = pd.DataFrame(lasso_results, columns = X.columns)
lasso_coef.head(10)

,wheelbase,enginesize,compressionratio,horsepower,peakrpm,citympg,highwaympg
0,174.698439,101.355892,310.832977,53.468711,1.807066,-102.823918,-0.000000
1,198.011655,95.626906,307.929517,49.578079,1.747472,-128.346491,-0.000000
2,135.120437,110.449859,369.194237,44.834411,1.683485,-99.427272,-20.805745
3,171.902893,116.043162,253.078426,47.107987,2.117826,-90.862698,0.000000
4,221.127545,107.137229,275.113540,54.438664,2.475482,-45.024131,-0.000000
5,158.906332,118.026381,306.183956,39.201271,2.116471,-299.527146,168.283217
6,163.214534,117.562814,284.627350,42.905403,2.346134,-112.959883,-0.000000
7,116.756862,116.093247,315.633127,51.372671,1.528826,-62.314109,-16.348285
8,193.588813,102.358029,333.717123,51.208097,2.516465,-105.216651,-0.000000
9,180.359114,111.166654,307.499118,46.925046,2.112494,-112.942350,-0.000000


In [5]:
len(lasso_coef.iloc[0,:])

7

In [6]:
len(lasso_coef.iloc[:,0])

1000

In [7]:
sum(lasso_coef.iloc[:,6] == 0)

762

In [8]:
## Checking Coefficient = 0 in the columns
for i in range(0, len(lasso_coef.iloc[0,:])):
    
    count = 0

    for j in range(0, len(lasso_coef.iloc[:,0])):
    
        if (lasso_coef.iloc[j, i] == 0):
            count = count + 1
            
    print("Column Index: ", i)
    print("Number of Coefficient = 0 in the column: ", count)
    print('')

Column Index:  0
Number of Coefficient = 0 in the column:  0

Column Index:  1
Number of Coefficient = 0 in the column:  0

Column Index:  2
Number of Coefficient = 0 in the column:  0

Column Index:  3
Number of Coefficient = 0 in the column:  0

Column Index:  4
Number of Coefficient = 0 in the column:  0

Column Index:  5
Number of Coefficient = 0 in the column:  12

Column Index:  6
Number of Coefficient = 0 in the column:  762



#### After going through all coefficients provided by the model LASSO, we can see that the column index 6, which is highwaympg, is not useful for our model. Therefore, it must be removed.

In [9]:
# (c)
X = X.drop('highwaympg', axis = 1)
X

,wheelbase,enginesize,compressionratio,horsepower,peakrpm,citympg
0,88.6,130,9.0,111,5000,21
1,88.6,130,9.0,111,5000,21
2,94.5,152,9.0,154,5000,19
3,99.8,109,10.0,102,5500,24
4,99.4,136,8.0,115,5500,18
...,...,...,...,...,...,...
200,109.1,141,9.5,114,5400,23
201,109.1,141,8.7,160,5300,19
202,109.1,173,8.8,134,5500,18
203,109.1,145,23.0,106,4800,26


In [10]:
# (d)
# Splitting the data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

# Normalizating input variables of the train and test dataset
def l2_normalization(X):
    
    x_mean = np.mean(X)
    l2 = np.sqrt(sum(X)**2)
    return (X - x_mean) / l2

X_train = X_train.apply(l2_normalization, axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)

In [39]:
# (e)
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV

# Building a multiple linear regression
lm_md = LinearRegression().fit(X_train, Y_train)

# Predicting on test dataset
md1_preds = lm_md.predict(X_test)

# Computing mse
linear_mse = np.mean(np.power(md1_preds - Y_test, 2))
linear_mse

17238816.525034405

In [36]:
# (f)
#    (i)
# Estimating optimal lambda with LASSO.
ridgeCV = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], cv = 5).fit(X_train, Y_train)
rigde_alpha = ridgeCV.alpha_
rigde_alpha

0.001

In [34]:
# (ii)
# Estimating optimal lambda with LASSO 100 times.
ridge_lambda = list()

for i in range(0, 100):
    ridgeCV = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], cv = 5).fit(X_train, Y_train)
    ridge_lambda.append(ridgeCV.alpha_)

ridge_results = pd.DataFrame(ridge_lambda)
ridge_results.value_counts()

0.001    100
dtype: int64

In [41]:
# Building Ridge Regression model
ridge_md = Ridge(alpha = rigde_alpha).fit(X_train, Y_train)

# Making Predictions made by Ridge Regression model
ridge_preds = ridge_md.predict(X_test)

# Evaluating the predictions using mse
ridge_mse = np.mean(np.power(ridge_preds - Y_test, 2))
ridge_mse

21828947.656970922

In [43]:
linear_mse - ridge_mse

-4590131.131936517

Based on the results from part (e) and (f), I would use the model from part (e), which is Linear Regression, to predict car prices.